Build model of energy metabolism described in Faizi et al 2018.

In [18]:
import tellurium as te
import math
import numpy as np
import matplotlib.pyplot as plt

In [26]:
model energy_metabolism

    compartment Cell, Environment;
    Cell = 1.0;
    Environment = 1.0;

    var Ci_ext;
    var Ci_int; 
    var Energy;
    var AminoAcid;
    var Transporter;
    var Metaenzyme;
    var Photoenzyme;
    var Ribosome;
    
    species Ci_ext, Ci_int, Energy, AminoAcid, Transporter, Metaenzyme, Photoenzyme, Ribosome;
    
    Ci_ext in cell; # extracellular inorganic carbon
    Ci_int in cell; # intracellular inorganic carbon
    Energy in cell; # energy units (here representing the ATP, NADH, etc pool)
    AminoAcid in cell; # amino acids
    Transporter in cell; # transporter protein for inorganic carbon
    Metaenzyme in cell; # metabolic multi-enzyme complex
    Photoenzyme in cell; # inactivated and activated photosynthetic machinery
    Ribosome in cell; # ribosome
    
    // Initializing variables
    Energy = 1.0e4
    AminoAcid = 1.0e4
    Ribosome = 1.0e4
    Transporter = 1.0e4
    Metaenzyme = 1.0e4
    Photoenzyme = 1.0e4 
    
    // Setting proportions of protein translation rates (beta_i > 0; sum(beta_i)<=1)    
    beta_R = 0.05
    beta_T = 0.25
    beta_M = 0.25
    beta_P = 0.025
    beta_J = 1.0;
 
    // Geometric Constants
    Perm_membrane = 0.0018; # permeability, dm min-1
    area_cell = 3.85e-9; # cell surface area, dm2 cell-1
    volume_cell = 2.24e-14; # L, cell-1
    N_a = 6.022e23; # Avogadro constant, mol-1
    
    // Kinetic Constants
    kcat_T = 726; # maximal import rate, min-1
    Km_T = 15; # Menten constant of T, uM
    kcat_M = 545; # maximal metabolic rate, min-1
    Km_M = 2441560; # Menten constant of M, molecules cell-1
    gamma_max = 1320; # maximal translation rate, aa min-1 molecules-1
    Km_AA = 10000; # Menten constant for aa, molecule cell-1
    Km_E = 10000; # Menten constant for e, molecule cell-1
    abs_xsec = 0.166; # effective absorption cross-section of P, m2 uE-1
    turnover_P = 1900; # maximal turnover rate of P, min-1
    k_d = 1.6e-6; # rate constant for photodamage
    K_i = 1.0e8; # product-inhibition constant for e synthesis, molecule cell-1

    // Material Constants 
    Dens_C = 1.4e10; # average cell density, aa cell-1
    n_R = 7358; # ribosome length, aa molecule-1
    n_T = 1681; # transporter length, aa molecule-1
    n_M = 28630; # metabolic enzyme complex length, aa molecule-1
    n_P = 95451; # photosynthetic enzyme complex length, aa molecule-1
    m_mu = 45; # e needed to create one aa
    m_gamma = 3; # e needed for one translational elongation step
    m_omega = 8; # e produced by Photosynthetic cycle
    m_c = 5; # average carbon chain length of aa
    sigma = 1;
    
    C_extracellular = 1000;
    C_intracellular = 1; 
    Ribosome = 1.0e5;
    Transporter = 1.0e4;
    Metaenzyme = 1.0e5;
    Photoenzyme = 1.0e5;
    Energy = 1.0e7;
    AminoAcid = 1.0e6;
    Irradiance = 1000; # ?
    
    gamma_max = 1320 #aa molecule^-1 min^-1
    K_E = 10000 # molecule cell^-1
    K_AA = 10000 # molecule cell^-1
    K_2 = 1; # ?

    // Assignment Rules
    gamma_R := beta_R * gamma_max / n_R * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid) 
    gamma_T := beta_T * gamma_max / n_T * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid) 
    gamma_M := beta_M * gamma_max / n_M * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid)
    gamma_P := beta_P * gamma_max / n_P * Ribosome * Energy / (Km_E + Energy) * AminoAcid / (Km_AA + AminoAcid) 
    
    v_t := Transporter * kcat_T * (Ci_ext/(Km_T+Ci_ext)) * (Energy/(Km_E+Energy)) # transport rate
    v_m := Metaenzyme + kcat_M * (Ci_int/(Km_M+Ci_int)) * (Energy/(Km_E+Energy)) # assimilation rate
    mu := (1/Dens_C) * (v_t/m_c) # growth rate
    gamma_j := beta_J * gamma_max/(n_R + n_T + n_M +n_P) * Ribosome * (Energy/(Km_E+Energy)) * (AminoAcid/(Km_AA+AminoAcid))
    v_2 := (K_2 * Energy * sigma * Irradiance * Photoenzyme)/(sigma * Irradiance + K_2 * Energy + mu)
    
    // Rate Laws
    Energy_P: -> Energy; m_omega * v_2 # energy produced
    Energy_C: Energy ->; v_t + m_mu * v_m + mu * Energy + m_gamma * (gamma_R*n_R + gamma_T*n_T + gamma_M*n_M + gamma_P*n_P) # energy consumed
    
    end 
    

Success: Model can be accessed via variable energy_metabolism


In [27]:
energy_metabolism.reset()
energy_metabolism.simulate(start = 0., end = 1.0e3, steps = 1000, selections=['time', 'Energy_P', 'Energy_C']) 
energy_metabolism.plot()